In [1]:
from __future__ import division
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import numpy as np
import cPickle, h5py, os, glob

from oc_ica import analysis, styles, plotting
reload(analysis)

Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 20.0% of memory, cuDNN 4007)


<module 'oc_ica.analysis' from '/home/jesse/Development/oc_ica/oc_ica/analysis.pyc'>

# HDF5 Storage

In [2]:
base_folder = '/home/jesse/Development/results/oc_ica/'

In [3]:
reload(analysis)
n_mixtures = 32
OCs = ['0.75', '1.0', '1.5', '2.0', '2.25', '2.5', '2.75', '3.0', '3.5', '4.0']
#OCs = ['1.0', '1.5']
k = 12
priors = ['COHERENCE_SOFT', 'COHERENCE', 'INIT']
keep_max = False

for OC in OCs:
    analysis.comparison_analysis_postprocess(base_folder, n_mixtures, OC, k, priors, keep_max, overwrite=True)
    
ks = [2, 4, 6, 8, 10, 12, 14, 16]
#ks = [2, 4]
OC = '2.0'
for k in ks:
    print 'kkkkkkkkkkkkkk', k
    analysis.comparison_analysis_postprocess(base_folder, n_mixtures, OC, k, priors, keep_max, overwrite=True)
    print 'kkkkkkkkkkkkkk', k

kkkkkkkkkkkkkk 2
kkkkkkkkkkkkkk 2
kkkkkkkkkkkkkk 4
kkkkkkkkkkkkkk 4
kkkkkkkkkkkkkk 6
kkkkkkkkkkkkkk 6
kkkkkkkkkkkkkk 8
kkkkkkkkkkkkkk 8
kkkkkkkkkkkkkk 10
kkkkkkkkkkkkkk 10
kkkkkkkkkkkkkk 12
kkkkkkkkkkkkkk 12
kkkkkkkkkkkkkk 14
kkkkkkkkkkkkkk 14
kkkkkkkkkkkkkk 16
kkkkkkkkkkkkkk 16


# Summary Plots

In [7]:
reload(analysis)
OC = '2.0'
k = 12

results, null_results, lambdas = analysis.comparison_analysis_postprocess(base_folder,
                                                                          n_mixtures, OC, k, priors, keep_max)


OCs = ['0.75', '1.0', '1.5', '2.0', '2.25', '2.5', '2.75', '3.0', '3.5', '4.0']
keep_OCs = ['1.0', '1.5', '2.0', '2.25', '2.5', '2.75', '3.0', '3.5']

#OCs = ['1.0', '1.5']
#keep_OCs = ['1.0', '1.5']
k = 12

models = ['2', '4', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F', 'SC', 'SM']
keep_models = ['2', '4', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F', 'SM', 'SC']

ks = [2, 4, 6, 8, 10, 12, 14, 16]
#ks = [2, 4]

labelpad = -0

n_prior = 0
f = plt.figure(figsize=(8, 5))
ae = plt.subplot2grid((2, 12), (0, 0), colspan=4)
am = plt.subplot2grid((2, 12), (0, 4), colspan=4)
ap = plt.subplot2grid((2, 12), (0, 8), colspan=4)
ax1 = plt.subplot2grid((2, 12), (1, 0), colspan=5)
ax2 = plt.subplot2grid((2, 12), (1, 5), colspan=5)

p = priors[n_prior]
print('prior: {}'.format(p))
ii = n_prior
for jj, m in enumerate(keep_models):
    jjp = models.index(m)
    if m in styles.models:
        fmt = styles.line_styles[m]

        color = styles.colors[m]
        label = styles.labels[m]

        if m == 'SM':
            delta = np.tile(np.nanmean(results[ii, jjp, 0, :, 0]), lambdas.size)
            mma = np.tile(np.nanmean(results[ii, jjp, 0, :, 1]), lambdas.size)
            null_delta = np.tile(np.nanmean(null_results[ii, jjp, 0, :, 0]), lambdas.size)
            null_mma = np.tile(np.nanmean(null_results[ii, jjp, 0, :, 1]), lambdas.size)
        else:
            delta = np.nanmean(results[ii, jjp, :, :, 0], axis=1)
            mma = np.nanmean(results[ii, jjp, :, :, 1], axis=1)
            null_delta = np.nanmean(null_results[ii, jjp, :, :, 0], axis=1)
            null_mma = np.nanmean(null_results[ii, jjp, :, :, 1], axis=1)
        ae.semilogx(lambdas, delta, fmt, label=label, c=color, lw=2,
                    path_effects=[pe.Stroke(linewidth=3, foreground='k'), pe.Normal()])
        
        ae.minorticks_off()
        am.semilogx(lambdas, mma, fmt, label=label, c=color, lw=2,
                    path_effects=[pe.Stroke(linewidth=3, foreground='k'), pe.Normal()])
        
        am.minorticks_off()
        ap.semilogx(lambdas, .5*(delta/null_delta + mma/null_mma), fmt, label=label, c=color, lw=2,
                    path_effects=[pe.Stroke(linewidth=3, foreground='k'), pe.Normal()])
        if m != 'SM':
            ae.semilogx(lambdas, delta, '.', c=color, ms=10, markeredgecolor='k')
            am.semilogx(lambdas, mma, '.', c=color, ms=10, markeredgecolor='k')
            ap.semilogx(lambdas, .5*(delta/null_delta + mma/null_mma), '.', c=color,
                        ms=10, markeredgecolor='k')
        
        ap.minorticks_off()
    
#ae.legend(loc='best', ncol=2, prop={'size': 8}, frameon=False)
#ae.set_title(p)
ae.set_ylabel(r'$\Delta P$', labelpad=labelpad, fontsize=12)
ae.set_ylim([0, 50])
#am.set_title(p)
am.set_ylabel(r'median($p_{\mathrm{min}}$)', labelpad=labelpad, fontsize=12)
am.set_ylim([0, 65])
#ap.set_title(p)
ap.set_ylabel('Normalized Mean', labelpad=labelpad, fontsize=12)
ap.set_ylim([0, 1.1])
    
ae.set_xlabel(r'$\lambda$', labelpad=labelpad, fontsize=12)
am.set_xlabel(r'$\lambda$', labelpad=labelpad, fontsize=12)
ap.set_xlabel(r'$\lambda$', labelpad=labelpad, fontsize=12)
ae.minorticks_off()
am.minorticks_off()
ap.minorticks_off()

x = np.array([float(oc) for oc in keep_OCs])
y = np.zeros((len(keep_models), x.size))
y_std = np.zeros_like(y)
for ii, OC in enumerate(keep_OCs):
    iik = OCs.index(OC)
    n_sources = int(n_mixtures * float(OC))
    results, null_results, lambdas = analysis.comparison_analysis_postprocess(base_folder,
                                                                          n_mixtures, OC, k, priors, keep_max)
    for kk, m in enumerate(keep_models):
        kkp = models.index(m)
        print ii, OC, kk, kkp, m, null_results.shape, results.shape
        if m == 'SM':
            mean_null = np.nanmean(null_results[n_prior, kkp, 0], axis=0, keepdims=True)
            r = results[n_prior, kkp, 0] / mean_null
            pos = np.nanmean(r, axis=0).sum()
            std = np.nanstd(r.sum(axis=-1))
        else:
            mean_null = np.nanmean(null_results[n_prior, kkp], axis=1, keepdims=True)
            r = results[n_prior, kkp] / mean_null
            r_mean = np.nanmean(r, axis=1).sum(axis=1)
            r_min_idx = r_mean.argmin()
            pos = np.nanmean(r[r_min_idx], axis=0).sum()
            std = np.nanstd(r[r_min_idx].sum(axis=-1))
        y[kk, ii] = pos
        y_std[kk, ii] = std

for ym, y_stdm, m in zip(y, y_std, keep_models):
    fmt = styles.line_styles[m]
    color = styles.colors[m]
    label = styles.labels[m]
    #ax1.errorbar(x, ym/2., yerr=y_stdm/2., fmt=fmt, color=color, label=label, lw=2,
    #             path_effects=[pe.Stroke(linewidth=3, foreground='k'), pe.Normal()])
    ax1.errorbar(x, ym/2., yerr=y_stdm/np.sqrt(10), fmt=fmt, color=color, label=label, lw=2,
             path_effects=[pe.Stroke(linewidth=3, foreground='k'), pe.Normal()])
ax1.set_xticks(np.linspace(1, 4, 4))
ax1.set_yticks(np.linspace(0, 2, 3))
ax1.set_ylim(0, 1)
ax1.set_xlim(.9, 3.6)
ax1.set_xlabel('Overcompleteness', labelpad=labelpad, fontsize=12)
ax1.set_ylabel('Normalized Mean', labelpad=labelpad, fontsize=12)


OC=2.
x = np.array(ks)
y = np.zeros((len(keep_models), x.size))
y_std = np.zeros_like(y)
for ii, k in enumerate(ks):
    n_sources = int(n_mixtures * float(OC))
    print ii, OC, kk, kkp, m, null_results.shape, results.shape
    results, null_results, lambdas = analysis.comparison_analysis_postprocess(base_folder,
                                                                          n_mixtures, OC, k, priors, keep_max)
    for kk, m in enumerate(keep_models):
        kkp = models.index(m)
        if m == 'SM':
            mean_null = np.nanmean(null_results[n_prior, kkp, 0], axis=0, keepdims=True)
            r = results[n_prior, kkp, 0] / mean_null
            pos = np.nanmean(r, axis=0).sum()
            std = np.nanstd(r.sum(axis=-1))
        else:
            mean_null = np.nanmean(null_results[n_prior, kkp], axis=1, keepdims=True)
            r = results[n_prior, kkp] / mean_null
            r_mean = np.nanmean(r, axis=1).sum(axis=1)
            r_min_idx = r_mean.argmin()
            pos = np.nanmean(r[r_min_idx], axis=0).sum()
            std = np.nanstd(r[r_min_idx].sum(axis=-1))
        y[kk, ii] = pos
        y_std[kk, ii] = std
handles = []
for ym, y_stdm, m in zip(y, y_std, keep_models):
    fmt = styles.line_styles[m]
    color = styles.colors[m]
    label = styles.labels[m]
    handles.append(ax2.errorbar(x, ym/2., yerr=y_stdm/np.sqrt(10), fmt=fmt, color=color, label=label, lw=2,
                  path_effects=[pe.Stroke(linewidth=3, foreground='k'), pe.Normal()])[0])
ax2.legend(loc='upper left', bbox_to_anchor=(.7, 1.15), frameon='False')
ax2.set_xlim(1.5, 16.5)
ax2.set_xticks(np.linspace(2, 16, 8))
ax2.set_yticks(np.linspace(0, 1, 2))
ax2.set_xlabel(r'$k$-sparseness', labelpad=labelpad, fontsize=12)
ax2.set_ylabel('Normalized Mean', labelpad=labelpad, fontsize=12)

f.text(.01, .95, 'a)', fontsize=14)
f.text(.33, .95, 'b)', fontsize=14)
f.text(.66, .95, 'c)', fontsize=14)
f.text(.01, .47, 'd)', fontsize=14)
f.text(.43, .47, 'e)', fontsize=14)
    
plt.tight_layout()
plt.savefig('/home/jesse/Downloads/figure3.pdf')
plt.show()

prior: COHERENCE_SOFT
0 1.0 0 0 2 (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
0 1.0 1 1 4 (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
0 1.0 2 2 COULOMB (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
0 1.0 3 3 COULOMB_F (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
0 1.0 4 4 RANDOM (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
0 1.0 5 5 RANDOM_F (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
0 1.0 6 7 SM (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
0 1.0 7 6 SC (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
1 1.5 0 0 2 (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
1 1.5 1 1 4 (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
1 1.5 2 2 COULOMB (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
1 1.5 3 3 COULOMB_F (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
1 1.5 4 4 RANDOM (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
1 1.5 5 5 RANDOM_F (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
1 1.5 6 7 SM (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
1 1.5 7 6 SC (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
2 2.0 0 0 2 (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
2 2.0 1 1 4 (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
2 2.0 2 2 COULOMB (3, 8, 17, 45, 2) (3, 8, 17, 10, 2)
2 2.

# A Check

In [3]:
a_file = 'a_array-32_OC-2.0_priors-COHERENCE_INIT.h5'
with h5py.File(os.path.join(base_folder, a_file), 'r') as f:
    A_array = f['A_array'].value
    A_priors = f['A_priors'].value

In [7]:
reload(plotting)
plotting.plot_angles_1column(np.transpose(A_array, (0, 1, 3, 2)), 1, A_priors, plot_init=False)
plt.show()

In [30]:
A_array.shape

(2, 10, 32, 64)

In [91]:
results.shape

(7, 11, 17, 10, 2)

In [92]:
results[0, -1]

array([[[ 54.        ,  37.72133636],
        [ 48.        ,  38.90277863],
        [ 52.        ,  37.9727478 ],
        [ 52.        ,  37.98392487],
        [ 56.        ,  38.73665619],
        [ 58.        ,  39.25117493],
        [ 56.        ,  38.4540863 ],
        [ 48.        ,  39.203125  ],
        [ 64.        ,  39.57582092],
        [ 52.        ,  38.31627655]],

       [[         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan]],

       [[         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        

In [104]:
rng = np.random.RandomState(202020)

lambd = 1.0
source_dim = 32
n_samples = 10000
k = 8

S = rng.laplace(0, lambd, size=(source_dim, n_samples))
S1 = np.copy(S)
S2 = np.copy(S)

for ii in range(n_samples):
    idxs = np.argsort(abs(S[:,ii]))[:-k]
    S1[idxs, ii] = 0.

    idxs = rng.permutation(source_dim)[:source_dim-k]
    S2[idxs, ii] = 0.

In [106]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 5))
ax1.hist(S1.ravel(), bins=1000)
ax1.set_yscale('log')
ax2.hist(S2.ravel(), bins=1000)
ax2.set_yscale('log')
plt.show()

In [70]:
S1.T

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , -4.84437276],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [71]:
S2.T

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])